# Tugas Praktikum

## Import Library

In [16]:
import pandas as pd
import datetime as dt

## Load Dataset

In [17]:
file_customers = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\customers.csv'
file_gasstations = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\gasstations.csv'
file_products = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\products.csv'
file_transactions = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\transactions_1k.csv'
file_yearmonth = r'D:\Kuliah\Semester2\Artificial_Intelligence\semua_kode\jobsheet_05\tugas_praktikum\sample_data\sample_data\yearmonth.csv'

## Preview Dataset

In [18]:
df_customers = pd.read_csv(file_customers)
df_customers.head()

,CustomerID,Segment,Currency
0,3,SME,EUR
1,5,LAM,EUR
2,6,SME,EUR
3,7,LAM,EUR
4,9,SME,EUR


In [19]:
df_gasstations = pd.read_csv(file_gasstations)
df_gasstations.head()

,GasStationID,ChainID,Country,Segment
0,44,13,CZE,Value for money
1,45,6,CZE,Premium
2,46,23,CZE,Other
3,47,33,CZE,Premium
4,48,4,CZE,Premium


In [20]:
df_products = pd.read_csv(file_products)
df_products.head()

,ProductID,Description
0,1,Rucní zadání
1,2,Nafta
2,3,Special
3,4,Super
4,5,Natural


In [21]:
df_transactions = pd.read_csv(file_transactions)
df_transactions.head()

,TransactionID,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price
0,1,2012-08-24,09:41:00,31543,486621,3704,2,28,672.64
1,2,2012-08-24,10:03:00,46707,550134,3704,2,18,430.72
2,3,2012-08-24,10:03:00,46707,550134,3704,23,1,121.99
3,4,2012-08-24,13:53:00,7654,684220,656,5,5,120.74
4,5,2012-08-24,08:49:00,17373,536109,741,2,28,645.05


In [22]:
df_yearmonth = pd.read_csv(file_yearmonth)
df_yearmonth.head()

,CustomerID,Date,Consumption
0,5,201207,528.30
1,5,201302,1598.28
2,5,201303,1931.36
3,5,201304,1497.14
4,6,201203,51.06


## Data Understanding

<li>Merge Tables</li>

In [23]:
# Merge transaksi dengan customers
df_merged = pd.merge(df_transactions, df_customers,
                     on='CustomerID', how='left')

# Merge dengan gasstations
df_merged = pd.merge(df_merged, df_gasstations, on='GasStationID', how='left')

# Merge dengan products
df_merged = pd.merge(df_merged, df_products, on='ProductID', how='left')

# Siapkan kolom 'Date' dari df_transactions agar cocok dengan df_yearmonth (yyyymm)
df_merged['Date'] = pd.to_datetime(df_merged['Date'])
df_merged['YearMonth'] = df_merged['Date'].dt.strftime('%Y%m')

# Ubah 'Date' di df_yearmonth jadi string
df_yearmonth['Date'] = df_yearmonth['Date'].astype(str)

# Merge dengan df_yearmonth berdasarkan CustomerID dan YearMonth
df_merged = pd.merge(
    df_merged,
    df_yearmonth,
    left_on=['CustomerID', 'YearMonth'],
    right_on=['CustomerID', 'Date'],
    how='left'
)

# Rapikan kolom: buang kolom Date duplikat dan kolom YearMonth
df_merged.drop(columns=['Date_y', 'YearMonth'], inplace=True)
df_merged.rename(columns={'Date_x': 'Date'}, inplace=True)

# Ubah kolom 'Date' menjadi object (string)
df_merged['Date'] = df_merged['Date'].astype(str)

df_merged.head()

,TransactionID,Date,Time,CustomerID,CardID,GasStationID,ProductID,Amount,Price,Segment_x,Currency,ChainID,Country,Segment_y,Description,Consumption
0,1,2012-08-24,09:41:00,31543,486621,3704,2,28,672.64,KAM,CZK,4,CZE,Premium,Nafta,142991.10
1,2,2012-08-24,10:03:00,46707,550134,3704,2,18,430.72,KAM,CZK,4,CZE,Premium,Nafta,123653.64
2,3,2012-08-24,10:03:00,46707,550134,3704,23,1,121.99,KAM,CZK,4,CZE,Premium,Provoz.nápl.,123653.64
3,4,2012-08-24,13:53:00,7654,684220,656,5,5,120.74,KAM,CZK,30,CZE,Noname,Natural,18107.97
4,5,2012-08-24,08:49:00,17373,536109,741,2,28,645.05,KAM,CZK,2,CZE,Noname,Nafta,72690.54


<li>Data Types</li>

In [24]:
df_merged.dtypes

TransactionID      int64
Date              object
Time              object
CustomerID         int64
CardID             int64
GasStationID       int64
ProductID          int64
Amount             int64
Price            float64
Segment_x         object
Currency          object
ChainID            int64
Country           object
Segment_y         object
Description       object
Consumption      float64
dtype: object

## Soal Nomer 1

### 1. Buatlah table list Top 5 Customers yang memiliki nilai transaksi paling banyak !

In [25]:
# Gabungkan berdasarkan CustomerID
merged = pd.merge(df_transactions, df_customers, on="CustomerID")

# Hitung jumlah transaksi dan total sales per CustomerID
summary = merged.groupby("CustomerID").agg(
    Amount=('TransactionID', 'count'),  # Banyaknya transaksi
    Sales=('Price', 'sum')              # Total uang yang dibelanjakan
).sort_values(by="Sales", ascending=False).head(5)

# Tampilkan dengan CustomerID sebagai index
summary.index.name = "Customer ID"
summary

,Amount,Sales
Customer ID,,
19182,19,8618.54
16358,5,8400.94
31543,18,6161.50
12459,10,5974.47
30895,10,5940.01


## Soal Nomer 2

### 2. Buatlah list Top 5 Gas stations yang memiliki nilai transaksi paling banyak !

In [26]:
# Gabungkan berdasarkan GasStationID
merged = pd.merge(df_transactions, df_gasstations, on="GasStationID")

# Hitung jumlah transaksi dan total sales per GasStationID
summary = merged.groupby("GasStationID").agg(
    Amount=('Amount', 'sum'),
    Sales=('Price', 'sum')
).sort_values(by="Sales", ascending=False).head(5)

# Tampilkan dengan GasStationID sebagai indeks
summary.index.name = "Gas Stations"
summary

,Amount,Sales
Gas Stations,,
4347,370,8557.94
48,295,6868.67
3250,320,6744.47
1155,292,6701.69
2417,273,6065.16


## Soal Nomer 3

### 3. Buatlah list Top 5 Jenis produk yang memiliki nilai transaksi paling banyak !

In [27]:
# Gabungkan transaksi dengan data produk
merged = pd.merge(df_transactions, df_products, on="ProductID")

# Hitung total Amount dan Sales per Produk
summary = merged.groupby("Description").agg(
    Amount=("Amount", "sum"),
    Sales=("Price", "sum"),
).sort_values(by="Sales", ascending=False).head(5)

# Tampilkan hasil
summary.index.name = "Products"
summary

,Amount,Sales
Products,,
Nafta,15230,345669.45
Diesel +,1120,27261.03
Natural,1045,24639.46
Nafta Plus,317,7233.72
Natural +,157,3999.92


## Soal Nomer 4

### 4. Buatlah deskripsi statistik untuk masing masing hari (23,24,25,26) pada dataset transaction_1k !

In [28]:
# Konversi kolom Date ke datetime
df_transactions["Date"] = pd.to_datetime(df_transactions["Date"])

# Filter data untuk tanggal 23–26
filtered_df = df_transactions[df_transactions["Date"].dt.day.isin([
                                                                  23, 24, 25, 26])]

# Definisikan fungsi statistik
stat_funcs = ['count', 'mean', 'std', 'min',
              lambda x: x.quantile(0.25),
              'median',
              lambda x: x.quantile(0.75),
              'max']

# Buat ringkasan statistik
summary = filtered_df.groupby("Date").agg({
    'Amount': stat_funcs,
    'Price': stat_funcs
})

# Format ulang nama kolom statistik
stat_names = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
summary.columns = pd.MultiIndex.from_tuples(
    [(col[0], stat_names[i % 8]) for i, col in enumerate(summary.columns)]
)

# Ganti nama kolom 'Price' menjadi 'Sales'
summary.rename(columns={"Price": "Sales"}, inplace=True)

# Tambahkan nama level kolom MultiIndex
summary.columns.names = ['Metric', 'Statistic']

# Tampilkan hasil
summary

Metric     Amount                                                    Sales  \
Statistic   count       mean        std min    25%   50%    75%  max count   
Date                                                                         
2012-08-23     10  29.600000  24.900692   1  21.25  26.5  35.25   91    10   
2012-08-24    492  19.936992  21.949630   0   5.00  19.0  27.00  264   492   
2012-08-25    425  18.880000  20.376114   0   4.00  17.0  26.00  154   425   
2012-08-26     73  21.219178  18.932810   0   6.00  20.0  29.00  101    73   

Metric                                                                        
Statistic         mean         std    min      25%     50%      75%      max  
Date                                                                          
2012-08-23  685.379000  534.046487  76.42  481.170  619.44  795.095  1994.71  
2012-08-24  409.939736  488.269481   1.76   76.125  332.45  615.395  5762.49  
2012-08-25  426.940071  453.134207   6.09  102.220  370.38  600.850  3600.84  
2012-08-26  487.431370  429.192499   4.74  142.840  437.03  675.520  2303.75

## Soal Nomer 5

### 5. Carilah waktu terbaik (hari dan jam) dimana paling banyak user gas station melakukan transaksi !

In [29]:
# Mengonversi kolom Amount dan Price ke numerik
df_transactions['Amount'] = pd.to_numeric(
    df_transactions['Amount'], errors='coerce')
df_transactions['Price'] = pd.to_numeric(
    df_transactions['Price'], errors='coerce')

# Menggabungkan Date dan Time menjadi satu kolom datetime
df_transactions['Datetime'] = pd.to_datetime(
    df_transactions['Date'].astype(
        str) + ' ' + df_transactions['Time'].astype(str),
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

# Mengekstrak tanggal dan jam
df_transactions['Date'] = df_transactions['Datetime'].dt.strftime('%Y-%m-%d')
# Jam dalam format 0-23
df_transactions['Hour'] = df_transactions['Datetime'].dt.hour

# Mengelompokkan data berdasarkan Date dan Hour, lalu menghitung Amount (sum) dan Sales (sum of Price)
result = df_transactions.groupby(['Date', 'Hour']).agg({
    'Amount': 'sum',  # Jumlah total Amount per jam
    'Price': 'sum'    # Total Price per jam
}).reset_index()

# Menampilkan hasil sebagai tabel DataFrame
result

,Date,Hour,Amount,Price
0,2012-08-23,9,79,1785.68
1,2012-08-23,20,26,720.89
2,2012-08-23,21,91,1994.71
3,2012-08-23,22,69,1603.51
4,2012-08-23,23,31,749.00
...,...,...,...,...
56,2012-08-26,9,310,7146.06
57,2012-08-26,10,120,2829.68
58,2012-08-26,11,67,1501.13
59,2012-08-26,12,152,3428.88


In [30]:
# Konversi kolom Amount dan Price ke numerik
df_transactions['Amount'] = pd.to_numeric(
    df_transactions['Amount'], errors='coerce')
df_transactions['Price'] = pd.to_numeric(
    df_transactions['Price'], errors='coerce')

# Gabungkan kolom Date dan Time menjadi kolom Datetime
df_transactions['Datetime'] = pd.to_datetime(
    df_transactions['Date'].astype(
        str) + ' ' + df_transactions['Time'].astype(str),
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

# Ekstrak Date dan Hour
df_transactions['Date'] = df_transactions['Datetime'].dt.strftime('%Y-%m-%d')
df_transactions['Hour'] = df_transactions['Datetime'].dt.hour

# Kelompokkan berdasarkan Date dan Hour, lalu hitung total Amount dan Price
grouped = df_transactions.groupby(['Date', 'Hour']).agg({
    'Amount': 'sum',
    'Price': 'sum'
}).reset_index()

# Ambil satu baris per Date dengan Amount terbesar (sesuai contohmu)
daily_max = grouped.loc[grouped.groupby('Date')['Amount'].idxmax()]

# Format ulang dan ubah nama kolom
daily_max = daily_max.rename(columns={'Hour': 'Time', 'Price': 'Gas Sales'})
daily_max['Gas Sales'] = daily_max['Gas Sales'].round(2)

# Jadikan kolom Date sebagai index
daily_max.set_index('Date', inplace=True)

# Tampilkan hasil
daily_max

,Time,Amount,Gas Sales
Date,,,
2012-08-23,21,91,1994.71
2012-08-24,16,1007,23528.20
2012-08-25,9,1034,19362.21
2012-08-26,8,348,7996.20


## Soal Nomer 6

### 6. Dari segi bisnis understanding, tujuan utama dari analisis data customer ini adalah? 

**Tujuan Utama dari Analisis Data Customer**

Berdasarkan analisis data terkait *Top Customers*, *Top Gas Stations*, *Top Products*, serta pola transaksi harian dan waktu terbaik pelanggan bertransaksi, berikut adalah tujuan bisnis utama dari analisis data customer:

1. **Mengidentifikasi pelanggan bernilai tinggi (high-value customers)**  
   ➤ Dari data Top 5 Customers, kita bisa mengetahui siapa saja pelanggan yang sering melakukan transaksi besar. Hal ini penting untuk memberikan layanan premium, promo eksklusif, atau loyalty rewards.

2. **Menganalisis performa lokasi (gas stations) terbaik**  
   ➤ Dengan mengetahui gas station dengan total nilai transaksi terbesar, perusahaan dapat memprioritaskan distribusi produk, promosi, atau bahkan ekspansi bisnis di area strategis.

3. **Mengenali produk paling laku**  
   ➤ Analisis jenis produk dengan nilai transaksi tertinggi membantu bisnis untuk fokus pada produk yang memiliki permintaan tinggi dan potensi profit besar.

4. **Mengoptimalkan waktu operasional dan promosi berdasarkan pola transaksi harian dan jam sibuk**  
   ➤ Dengan memahami jam dan hari paling sibuk dari aktivitas pelanggan, bisnis dapat mengatur stok, tenaga kerja, serta mengadakan promosi tepat waktu untuk memaksimalkan penjualan.

5. **Segmentasi pelanggan berdasarkan perilaku transaksi**  
   ➤ Contohnya, pelanggan yang sering membeli dalam jumlah kecil namun sering, berbeda dengan pelanggan yang beli dalam jumlah besar meski jarang. Strategi pendekatan bisa disesuaikan.

6. **Meningkatkan retensi dan loyalitas pelanggan**  
   ➤ Dengan memahami pola dan kebiasaan pelanggan bernilai tinggi, perusahaan bisa menciptakan strategi yang membuat mereka tetap loyal, misalnya dengan program loyalti, diskon personal, atau reward berjenjang.

7. **Prediksi nilai jangka panjang pelanggan (Customer Lifetime Value)**  
   ➤ Informasi dari semua analisis di atas dapat digunakan untuk memproyeksikan nilai jangka panjang seorang pelanggan terhadap bisnis.

Singkatnya, analisis data customer ini bertujuan untuk **mengambil keputusan bisnis yang lebih akurat dan strategis** dalam rangka meningkatkan pendapatan, efisiensi operasional, dan kepuasan pelanggan.
